In [ ]:

import pandas as pd
import numpy as np
import zipfile


df= pd.read_csv('/content/processed_data.csv')


from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from statistics import mean




columns = df.columns.tolist()
columns.remove('sii')
columns.remove('id')


#splitting dataset into X and y
X = df[columns]
y = df['sii']


#splitting dataset on convetional 80%, 20% train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)




In [ ]:
#Linear Regression Model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)


print('Training score: {}'.format(lr_model.score(X_train, y_train)))

print('Test score: {}'.format(lr_model.score(X_test, y_test)))

#predicting
y_pred = lr_model.predict(X_test)

#reporting mse
mse = mean_squared_error(y_test, y_pred)

print(mse)


Training score: 0.3686945082115627
Test score: 0.22970465538613194
0.19945147315894798


In [ ]:



# List to maintain the different values of alpha for ridge regularization
alpha_ridge = []

# List to maintain the different cross vals for each alpha
cross_val_scores_ridge = []

# Loop to compute the different values of cross-validation scores
for i in range(1, 10**2):
    ridgeModel = Ridge(alpha = i * 0.01)
    ridgeModel.fit(X_train, y_train)
    scores = cross_val_score(ridgeModel, X, y, cv = 10)
    avg_cross_val_score = mean(scores)*100
    cross_val_scores_ridge.append(avg_cross_val_score)
    alpha_ridge.append(i * 0.01)



# List to maintain the different values of alpha for lasso regularization
alpha_lasso = []

# List to maintain the different cross vals for each alpha
cross_val_scores_lasso = []

# Loop to compute the different values of cross-validation scores
for i in range(1, 10**2):
    lassoModel = Lasso(alpha = i * 0.01)
    lassoModel.fit(X_train, y_train)
    scores = cross_val_score(lassoModel, X, y, cv = 10)
    avg_cross_val_score = mean(scores)*100
    cross_val_scores_lasso.append(avg_cross_val_score)
    alpha_lasso.append(i * 0.01)


#grabbing index of alpha with highest cross val
best_alpha_ridge_i = cross_val_scores_ridge.index(max(cross_val_scores_ridge))
best_alpha_lasso_i = cross_val_scores_lasso.index(max(cross_val_scores_lasso))

#printing best alpha along with its cross val value
print("alpha_ridge", alpha_ridge[best_alpha_ridge_i], "crossval", cross_val_scores_ridge[best_alpha_ridge_i])
print("alpha_lasso", alpha_lasso[best_alpha_lasso_i], "crossval", cross_val_scores_lasso[best_alpha_lasso_i])

alpha_ridge 0.99 crossval 27.826609439588008
alpha_lasso 0.01 crossval 29.728104204913937
